In [2]:
%matplotlib qt
#to open matplotlib window

import csv
import matplotlib.pyplot as plt
import numpy as np
import datetime
import paramiko
import os
import time

PORT = 22
USERNAME = "root"
PASSWORD = "root"
REMOTE_PATH = "Pitaya-Tests/" 
REMOTE_FOLDER = "Pitaya-Tests"
SAMPLING_RATE = 125e+6

hostName = "169.254.29.18"
port = 22


def run_acquisition_command(samplesNb, dec,FidNb, FileName, larmorFrequency, excitationDuration, delayRepeat): #voir si file path marche    
    filePath = "mesures/" + FileName
    command = f"cd {REMOTE_FOLDER} && ./Acquisition_axi.exe {samplesNb} {dec} {FidNb} {filePath} {larmorFrequency} {excitationDuration} {delayRepeat}"
    stdin, stdout, stderr = client.exec_command(command)
    output = stdout.read().decode()
    errors = stderr.read().decode()
    print(f"[CMD-SSH] {command}")
    if output:
        print("[OUTPUT SSH]\n", output)
    if errors:
        print("[ERROR SHH]\n", errors)

def create_file_wdate(nameFile):
    # Créer le dossier local avec timestamp
    now = datetime.datetime.now()
    name_local_file = f"mesures/{nameFile}-{now.strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(name_local_file, exist_ok=True)
    
    return name_local_file 
    
def download_file_sftp(nameRemoteFile,nameRemoteFolder,nameLocalFolder):
    """Télécharge le fichier CSV via SFTP"""
    
    # Téléchargement du fichier
    remote_path = REMOTE_PATH + nameRemoteFolder+'/' + nameRemoteFile
    local_path = os.path.join(nameLocalFolder, nameRemoteFile)
    
    try:
        sftp.get(remote_path, local_path)
        print(f"Téléchargé: {nameRemoteFile}")
    except FileNotFoundError:
        print(f"Fichier non trouvé: {remote_path}")
    except Exception as e:
        print(f"Erreur lors du téléchargement de {nameRemoteFile}: {e}")

def read_file(fileName,folderName):
        
        pathOfFile = os.path.join(folderName,fileName)
        "open the file at the given path and give back the numpy vectors time, voltage"
        voltage_acc = []

        with open(pathOfFile, 'r', encoding='utf-8') as fichier_:
            lecteur = csv.reader(fichier_)

            # Lire et parser l'entête du fichier
            ligne_entete = next(lecteur)
            dsize = int(ligne_entete[0])
            decimation = int(ligne_entete[1])
            nombre_de_FID = int(ligne_entete[2])
            gain = float(ligne_entete[3])
            offset = float(ligne_entete[4])
            nb_bits = int(ligne_entete[5])

            # Initialisation
            voltage = [[] for _ in range(nombre_de_FID)]
            mean = []
            voltage_acc = np.zeros(dsize)  # Initialiser au bon format et taille

            # Lecture et conversion des tensions
            for j in range(nombre_de_FID):
                ligne = next(lecteur)

                signal = []

                # for val_bin in ligne:
                #     #val_bin = int(val_bin)
                #     val = convert_to_volt(val_bin, nb_bits, gain, offset)
                #     signal.append(val)  # pas besoin de np.append ici


                for val in ligne:
                    signal.append(float(val))

                # Convertir la ligne en tableau numpy
                signal = np.array(signal)

                #### Calcul de la moyenne et centrage
                #moyenne = np.mean(signal)
                #signal_centre = signal - moyenne

                # Stocker les données
                voltage[j] = signal
                #mean.append(moyenne)

                # Accumulation du signal total
                voltage_acc += signal

        # Création du tableau temps
        duree_mesure = (dsize * decimation) / SAMPLING_RATE 
        time = np.linspace(0, duree_mesure, dsize, endpoint=False)

        # Calcul de la moyenn et centrage du signal accumulé
        moyenne = np.mean(voltage_acc[-100000:-1000])
        voltage_acc = voltage_acc - moyenne

        print(f"Fichier {pathOfFile} lu. {len(voltage)} signaux FID chargés.")
        return time, voltage, voltage_acc

def plot_acc(graph_name, time_axis, voltage_matrix, voltage_accumulated_axis):
    
    amountFID = len(voltage_matrix)

    plt.figure(figsize=(12, 7))

    # Affichage des courbes FID
    for w in range(amountFID):
        plt.plot(time_axis, voltage_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')

    # Affichage du signal accumulé
    plt.plot(time_axis, voltage_accumulated_axis, marker='+', linestyle='-', label='Total', linewidth=2, color='black')

    # Mise en forme du graphique
    plt.title(f'Superposition - {graph_name}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Tension (V)')
    plt.grid(False, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

def subpolts_acc(graph_name, time_axis, voltage_matrix, voltage_accumulated_axis):
    # Supposons que voltage_matrix et time_axis soient déjà définis
    amountFID = len(voltage_matrix)

    # Créer une figure et un ensemble de sous-graphes
    fig, axs = plt.subplots(amountFID, 1, figsize=(12, 7 * amountFID), sharex=True)

    # Affichage des courbes FID sur des sous-graphes séparés
    for w in range(amountFID):
        axs[w].plot(time_axis, voltage_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')
        axs[w].set_title(f'FID {w+1}')
        axs[w].set_ylabel('Tension (V)')
        axs[w].grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
        axs[w].legend(loc='upper right')

    # Mise en forme globale du graphique
    fig.suptitle(f'Superposition - {graph_name}', y=1.02)  # y est légèrement ajusté pour éviter le chevauchement
    plt.xlabel('Temps (s)')
    plt.tight_layout()
    plt.show()

def plot_fourier_transform(time, voltage):
    # Ensure time and voltage are numpy arrays
    time = np.array(time)
    voltage = np.array(voltage)

    # Sampling interval and frequency
    dt = time[1] - time[0]
    fs = 1 / dt

    # Compute FFT
    N = len(voltage)
    fft_values = np.fft.fft(voltage)
    freq = np.fft.fftfreq(N, dt)

    # Keep only the positive frequencies
    #idx = np.where(freq >= 0)
    #freq = freq[idx]
    magnitude = np.abs(fft_values) * 2 / N  # Normalize amplitude

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(freq, magnitude)
    plt.title("Fourier Transform")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [9]:
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostName, username=USERNAME, password=PASSWORD,port=port)
print(f"[INFO] Connexion SSH établie à {hostName}")

transport = paramiko.Transport((hostName, PORT))
transport.connect(username=USERNAME, password=PASSWORD)
sftp = paramiko.SFTPClient.from_transport(transport)
print(f"[INFO] Connexion SFTP établie à {hostName}")

[INFO] Connexion SSH établie à 169.254.29.18
[INFO] Connexion SFTP établie à 169.254.29.18


In [ ]:
sample_Amount = 2720               #Number of points measured (MAX = 524288) must be a multiple of 2
decimation = 1216                    #Decimation
aquisiton_Amount = 10               #Nubmer of acquisitons
experiment_name = "test.csv"            #name of the file of the measured tension
larmor_Frequency_Hertz = 13.4e+6          #Larmor frequency
excitation_duration_seconds = 50.000e-6     #Excitation time
delay_repeat_seconds = 5


total_time = (sample_Amount * decimation)/SAMPLING_RATE
print(f"temps mesuré : {total_time}s")
print(f"temps total : {total_time*aquisiton_Amount}")

nb_cycles = larmor_Frequency_Hertz*excitation_duration_seconds
print(f"nb cycles burst : {nb_cycles}")

temps mesuré : 0.009961472s
temps total : 0.029884416000000004
nb cycles burst : 75.4602


In [11]:
run_acquisition_command(sample_Amount, 
                        decimation, 
                        aquisiton_Amount, 
                        experiment_name, 
                        larmor_Frequency_Hertz, 
                        excitation_duration_seconds,
                        delay_repeat_seconds)


[CMD-SSH] cd Pitaya-Tests && ./Acquisition_axi.exe 1024 1216 3 mesures/test.csv 2434.2 0.031 1
[OUTPUT SSH]
 larmor 2434.199951, duration excitation 0.031000
fichier crée : mesures/test.csv
wait to be filled
Tr pos1: 0x0
ecriture FID 0
wait to be filled
Tr pos1: 0x0
ecriture FID 1
wait to be filled
Tr pos1: 0x0
ecriture FID 2
Temps d'execution : 0.038821



In [ ]:
nameRemoteFile = experiment_name
nameRemoteFolder = "mesures"
nameLocalFolder = create_file_wdate("FindFreq")
download_file_sftp(nameRemoteFile,nameRemoteFolder,nameLocalFolder)

time_array, voltage_array_matrix, voltageAcc_array = read_file(nameRemoteFile,nameLocalFolder)
plot_acc(experiment_name, time_array, voltage_array_matrix, voltageAcc_array)
plot_fourier_transform(time_array, voltageAcc_array)

Téléchargé: test.csv
Fichier mesures/Acquisition_test-20251008_104523\test.csv lu. 3 signaux FID chargés.


C:\Users\page\AppData\Local\Temp\ipykernel_21400\1005802505.py:138: UserWarning: First parameter to grid() is false, but line properties are supplied. The grid will be enabled.
  plt.grid(False, which='both', linestyle='--', linewidth=0.5, alpha=0.7)


In [63]:
client.close()
# Fermer la connexion
sftp.close()
transport.close()
print("Connexion SFTP fermée")

Connexion SFTP fermée


Socket exception: Une connexion existante a dû être fermée par l’hôte distant (10054)
Socket exception: Une connexion existante a dû être fermée par l’hôte distant (10054)
Socket exception: Une connexion existante a dû être fermée par l’hôte distant (10054)
Socket exception: Une connexion existante a dû être fermée par l’hôte distant (10054)


In [ ]:
nameLocalFile = "balayage-20250916_104324"
nameLocalFolder = "balayage_time.csv"

time_array, voltage_array_matrix, voltageAcc_array = read_file(nameLocalFile,nameLocalFolder)
plot_acc(experiment_name, time_array, voltage_array_matrix, voltageAcc_array)
plot_fourier_transform(time_array, voltageAcc_array)